# First steps with Prophet and NeuralProphet for forecasting

In [ ]:
!pip install tqdm==4.54.1 # force specific version, because newer version does not seem to work with neuralprophet...

In [ ]:
# install neuralprophet first
!pip install neuralprophet

In [ ]:
# packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly # for interactive plots

from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed
# documentation: https://ourownstory.github.io/neural_prophet/

# Load and convert data

In [ ]:
# load data
df = pd.read_csv('../input/covid19-data-from-john-hopkins-university/CONVENIENT_global_deaths.csv')

In [ ]:
# extract time series
sel_country = 'Italy'
n = df.shape[0] - 1
df_sel = df[1:n+1] # remove first row!
df_sel = df_sel[['Country/Region',sel_country]] # select date + specific country column
# change column names
df_sel = df_sel.rename(columns={'Country/Region': 'ds', sel_country : 'y'})

df_sel.tail()

In [ ]:
# plot original time series
plt.rcParams['figure.figsize'] = (16, 8) # resize plot window

df_sel.plot(x='ds', y='y');
plt.title('Development of Deaths')
plt.grid()
plt.show()

# Use Prophet

In [ ]:
# define a fit model
m = Prophet(n_changepoints=150,
            changepoint_prior_scale=0.5,
            changepoint_range=0.95,
            weekly_seasonality=7)
m.fit(df_sel);

In [ ]:
# number of days to project into the future
n_projection = 28
# prepare data frame to include future days for prediction
future = m.make_future_dataframe(periods=n_projection)

In [ ]:
# calc predictions
forecast_1 = m.predict(future)
# and show result
forecast_1.head()
# and plot
m.plot(forecast_1);

In [ ]:
# show components of fit
m.plot_components(forecast_1);

In [ ]:
# interactive version of plot
plot_plotly(m, forecast_1)

# Now use NeuralProphet

In [ ]:
# define and fit model
set_random_seed(1234)
nm = NeuralProphet(n_changepoints=100,
                   daily_seasonality=False,                   
                   yearly_seasonality=False,
                   weekly_seasonality=7,
                   changepoints_range=0.95,
                   num_hidden_layers=1, # default: 0
                   seasonality_reg=0.1)

nm_result = nm.fit(df_sel, freq='D')

In [ ]:
nm_result.head()

In [ ]:
plt.rcParams['figure.figsize'] = (7, 9) # resize plot window
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True)
ax1.plot(nm_result.SmoothL1Loss)
ax1.set_title('SmoothL1Loss')
ax1.grid()
ax2.plot(nm_result.MAE)
ax2.set_title('MAE')
ax2.grid()
ax3.plot(nm_result.RegLoss)
ax3.set_title('RegLoss')
ax3.grid()
plt.show()

In [ ]:
# prepare data frame to include future days for prediction
future = nm.make_future_dataframe(df_sel, periods = n_projection,
                                  n_historic_predictions=len(df_sel))
# predict
forecast_2 = nm.predict(future)
# and plot forecast
nm.plot(forecast_2);

In [ ]:
# show components of fit again
nm.plot_components(forecast_2);

In [ ]:
# detailed parameters
nm.plot_parameters();